In [1]:
#writing ensight Gold binary (comaptible with vtk 8)
import numpy as np 
import os
from sys import getsizeof
import pandas


In [2]:
inputfolder = '../'
project_name = 'EP_141'
outputfolder = './'

alya_id_type = np.int64
ensight_id_type = np.int32
ensight_float_type = np.float32
iterationid_number_of_digits = 6  #how many digits to use for the iteration id in variable files



# Functions

In [3]:
def identify_alya_id_type(project_name):
    #read he header and see if its int8 or int4
    filename = os.path.join(inputfolder,f'{project_name}-LNODS.post.alyabin');
    with open(filename, 'rb') as f:
        header = read_header(f)
        if header['strings'][6] == '4BYTE':
            alya_id_type = np.int32
        elif header['strings'][6] == '8BYTE':
            alya_id_type = np.int64
        else:
            assert False, f'Alya id type {header[6]} is not supported'
    return alya_id_type        

In [4]:
def read_one_fp90_record(file_object, number_of_elements, datatype):
    #fortran stores length with every block, in the beginning and the end
    count_read = 0
    record = []
    while count_read<number_of_elements:
        #in case the record is stored as several blocks join them
        block_len = np.fromfile(file_object, dtype=np.int32, count=1)
        #block_len is in bytes
        block = np.fromfile(file_object, dtype=datatype, count=block_len[0]//np.dtype(datatype).itemsize)
        block_len = np.fromfile(file_object, dtype=np.int32, count=1)    
        count_read = count_read+block_len
        record = record + [block]
        
    return np.concatenate(record)

In [5]:
def read_header(file_object):
    #a sanity check
    assert hasattr(file_object, 'read'), "read_header: argument is not a file object"
    
    ihead = read_one_fp90_record(file_object, 1, np.int32)    #! Header: 1234, int32
    assert ihead[0] ==1234, "Header is not 1234"
    strings = [];
    integers = [];
    for i in range(0,9):
        strings = strings +[read_one_fp90_record(file_object, 8, np.uint8).tostring().decode().strip()]   
        #read(ii) strings(1) ! AlyaPost, char8bytes
        #read(ii) strings(2) ! Version, char8bytes
        #read(ii) strings(3) ! NAME, char8bytes
        #read(ii) strings(4) ! SCALA/VECTO, char8bytes
        #read(ii) strings(5) ! NELEM/NPOIN/NBOUN, char8bytes
        #read(ii) strings(6) ! INTEG/REAL,char8bytes
        #read(ii) strings(7) ! 4BYTE/8BYTE, char8bytes -- 4/8 byte integers used subsequently for ids/element type
        #read(ii) strings(8) ! SEQUE/PARAL, char8bytes      
        #read(ii) strings(9) ! NOFIL/FILTE, char8bytes
        
    #for i in range(len(strings)):
    #    print(strings[i])
        
        
    for i in range(0,5):
        integers = integers +[read_one_fp90_record(file_object, 1, np.int32)]   

    #for i in range(len(integers)):
    #    print(integers[i])

    
        
    #read(ii) integers(1) ! ??? int32
    #read(ii) integers(2) ! nelem_total, int32
    #read(ii) integers(3) ! # Subdomains, number of parts?
    if( strings[1][0:5] != 'V0001' ):
        integers = integers +[read_one_fp90_record(file_object, 1, np.int32)]   
        #read(ii) integers(4) ! Time step, int32
        
    reals = read_one_fp90_record(file_object, 1, np.float64)
    #read(ii) reals(1)    ! Time, float64

    if( strings[1][0:5] == 'V0001' ):
        integers[3] = int(reals)  #! floor()?

    return {'strings':strings, 'integers':integers, 'reals':reals}

In [6]:
def read_alya_array(filename, number_of_blocks, datatype):
    with open(filename,'rb') as f:
        header = read_header(f)
        number_of_dimensions = header['integers'][0][0]
        number_of_tuples_total = header['integers'][1][0]
        time_instant_int = header['integers'][3][0]
        time_instant_real = header['reals'][0]
        print(f'Reading array: {number_of_dimensions} dim, {number_of_tuples_total} tuples\n')

        tuples = np.zeros((number_of_tuples_total,number_of_dimensions), dtype=datatype)

        c = 0;
        tuples_per_block = np.zeros(number_of_blocks, dtype=np.int32)
        
        for i in range(number_of_blocks):
            number_of_tuples_in_block = read_one_fp90_record(f, 1, alya_id_type)[0] #stored by alya
            tuples_per_block[i] = number_of_tuples_in_block

            print(f'Block {i}/{number_of_blocks}: {(number_of_tuples_in_block)} tuples\n')
            tuples_temp = read_one_fp90_record(f, number_of_dimensions*number_of_tuples_in_block, datatype)
            
            tuples[c:c+number_of_tuples_in_block, :] =                 np.reshape(tuples_temp, (number_of_tuples_in_block,number_of_dimensions))
            c = c+number_of_tuples_in_block

    return {'tuples':tuples, 'time_real':time_instant_real, 'time_int':time_instant_int, 'tuples_per_block':tuples_per_block};

In [7]:
def read_alya_variable(variable_name, iteration, number_of_blocks):
    field_filename = os.path.join(inputfolder, '%s-%s-%08d.post.alyabin'% (project_name, variable_name, iteration)) 
    print(field_filename)
    
    field_dtype = alya_id_type
    association = '';
    variabletype = ''; #scalar, vector, ...

    with open(field_filename,'rb') as f:
        header = read_header(f)


    
    if( header['strings'][5] == 'REAL' ):
        field_dtype = np.float64
    if( header['strings'][5] == 'INTEG' ):
        field_dtype = alya_id_type

    if( header['strings'][4] == 'NPOIN' ):
        association = 'node'
    else:
        association = 'element'


    if( header['strings'][3] == 'SCALA' ):
        variabletype = 'scalar'
    elif( header['strings'][3] == 'VECTO' ):
        variabletype = 'vector'
        print('Reading vectors, this has not been tested yet')
    else:
        assert False, "unsupported type of variable"


    if( header['strings'][8] == 'NOFIL' ):
        field_data = read_alya_array(field_filename, number_of_blocks, field_dtype)
    else: 
        assert False, "Filtered types not supported"
        
        
    return {'values':field_data, 'association':association, 'variabletype':variabletype}

In [8]:
def write_geometry(project_name, number_of_blocks):
    point_coordinates = read_alya_array(os.path.join(inputfolder,f'{project_name}-COORD.post.alyabin'), \
                                        number_of_blocks, np.float64)
    element_types = read_alya_array(os.path.join(inputfolder,f'{project_name}-LTYPE.post.alyabin'),  \
                                    number_of_blocks, alya_id_type)
    #Read connectivity (indices inside start with 1)
    connectivity = read_alya_array(os.path.join(inputfolder,f'{project_name}-LNODS.post.alyabin'),    \
                                   number_of_blocks, alya_id_type)

    #elements have ids local to each block, tranform them to global ids
    a = connectivity['tuples_per_block'][0]
    npts =  point_coordinates['tuples_per_block'][0]
    for i in range(1,connectivity['tuples_per_block'].shape[0]): #for each block, skip 0
        b = a + connectivity['tuples_per_block'][i]
        connectivity['tuples'][a:b,:] = connectivity['tuples'][a:b,:] + npts
        a = b
        npts = npts + point_coordinates['tuples_per_block'][i]
        
        
    #assume all elements are the same
    element_type = b'hexa8';
    if element_types['tuples'][0] == 37: #alya hex08 element
        element_type = b'hexa8';
    elif element_types['tuples'][0] == 30: #alya tet04 element
        element_type = b'tetra4';
        
    #geometry ensight
    with open(os.path.join(outputfolder,f'{project_name}.ensi.geo'),'wb') as f:
        f.write(b'C Binary'.ljust(80))
        f.write(b'description line 1'.ljust(80))
        f.write(b'description line 2'.ljust(80))
        f.write(b'node id given'.ljust(80))
        f.write(b'element id given'.ljust(80))
        f.write(b'part'.ljust(80))
        f.write(np.array([1], dtype=ensight_id_type))   #int
        f.write(b'description line 1'.ljust(80))
        f.write(b'coordinates'.ljust(80))

        number_of_points = point_coordinates['tuples'].shape[0]
        f.write(np.array([number_of_points], dtype=ensight_id_type))   #int
        f.write(np.arange(1,number_of_points+1, dtype=ensight_id_type))
        f.write( point_coordinates['tuples'][:,0].ravel().astype(ensight_float_type) )  #x coord
        f.write( point_coordinates['tuples'][:,1].ravel().astype(ensight_float_type) )  #y coord
        f.write( point_coordinates['tuples'][:,2].ravel().astype(ensight_float_type) )  #z coord

        f.write(element_type.ljust(80))  #tetra4 or hexa8
        number_of_elements = connectivity['tuples'].shape[0]
        f.write(np.array([number_of_elements], dtype=ensight_id_type))   #int
        f.write(np.arange(1,number_of_elements+1, dtype=ensight_id_type))
        f.write(connectivity['tuples'].ravel().astype(ensight_id_type))



In [9]:
def write_variable(varname, iteration, number_of_blocks):
    data = read_alya_variable(varname, iteration, number_of_blocks)

    
    #variable ensight
    fmt = '%s.ensi.%s-'+f'%0{iterationid_number_of_digits}d';
    with open( os.path.join(outputfolder, fmt % (project_name, varname, iteration+1)),'wb') as f:
        f.write(b'description line 1'.ljust(80))
        f.write(b'part'.ljust(80))
        f.write(np.array([1], dtype=ensight_id_type))   #int
        f.write(b'coordinates'.ljust(80))

        if data['variabletype']=='scalar':
            f.write( data['values']['tuples'].ravel().astype(ensight_float_type) )  #z coord    
        elif data['variabletype']=='vector':
            #data has coordinates in the order [[x,y,z],[x,y,z],...]
            #expected order of coordinates
            #vx_n1 vx_n2 ... vx_nn nn floats
            #vy_n1 vy_n2 ... vy_nn nn floats
            #vz_n1 vz_n2 ... vz_nn nn floats
            #Rearrange the  matrix
            f.write( data['values']['tuples'].ravel(order='F').astype(ensight_float_type) )  #z coord    
        else:
            assert False, f"Unknown varibale type: {data['variabletype']}"
        
        
        
        
    return {'time_real':data['values']['time_real'], 'time_int':data['values']['time_int'],             'variable_type':data['variabletype'], 'variable_association':data['association']}


# Main program

In [10]:
#identify id type
alya_id_type = identify_alya_id_type(project_name)
print(f'Using Alya id type {alya_id_type}')

Using Alya id type <class 'numpy.int64'>


# Read the partitioning info

In [11]:
#read the partitioning info
partition_filename = os.path.join(inputfolder,f'{project_name}.post.alyapar')
with open(partition_filename) as f:
    partitions = np.fromstring(f.read(), dtype=alya_id_type, sep=' ')

#describes the mesh partitions
#partition_id,  NumberOfElementsInPartition,  NumberOfPointsInPartition, NumberOfBoundariesInPartition
partitions = np.reshape(partitions[1:],(partitions[0],4))
partitions

array([[     1, 331268,  61061,      0],
       [     2, 329190,  60172,      0],
       [     3, 329175,  60531,      0],
       [     4, 329253,  59714,      0],
       [     5, 329374,  59787,      0],
       [     6, 329263,  60321,      0],
       [     7, 328741,  62375,      0],
       [     8, 328823,  60621,      0],
       [     9, 328795,  60369,      0],
       [    10, 328820,  61100,      0],
       [    11, 328918,  62291,      0],
       [    12, 328765,  62713,      0],
       [    13, 329613,  60751,      0],
       [    14, 328939,  59882,      0],
       [    15, 328946,  59789,      0],
       [    16, 332351,  60555,      0],
       [    17, 328954,  60170,      0],
       [    18, 329261,  60319,      0],
       [    19, 329143,  59632,      0],
       [    20, 329137,  59482,      0],
       [    21, 329165,  59792,      0],
       [    22, 329261,  59544,      0],
       [    23, 329261,  59756,      0],
       [    24, 324450,  58545,      0],
       [    25, 

In [12]:
#this script does not handle boundaries yet
assert (partitions[:,3]==0).all(),  'this script does not handle boundaries yet'

# Identify variables

In [13]:
#Parse the filelist of the fields
with open(os.path.join(inputfolder, f'{project_name}.post.alyafil'),'r') as f:
    field_filelist = f.read().splitlines()
    
#remove spaces and empty lines
field_filelist = [x.strip() for x in field_filelist if x.strip()!='']

#extract array names and iteration numbers 
fields = []
iteration_numbers = []
for filename in field_filelist:
    s1 = filename.split('-');
    fields = fields + [s1[1]]
    iteration_numbers =  iteration_numbers + [ int(s1[2].split('.')[0]) ] #this will be long in python 3
    
variable_info = pandas.DataFrame({'field':fields, 'iteration':iteration_numbers,'filename':field_filelist})
variable_info['time_int'] = 0
variable_info['time_real'] = 0
variable_info['variabletype']=''
variable_info['association']=''
variable_info

,field,filename,iteration,time_int,time_real,variabletype,association
0,INTRA,EP_141-INTRA-00000000.post.alyabin,0,0,0,,


# Unknown stuff

In [30]:
#god knows what are these
#LNINV = read_alya_array(os.path.join(inputfolder,f'{project_name}-LNINV.post.alyabin'), \
#                                number_of_blocks, alya_id_type)
#LELCH = read_alya_array(os.path.join(inputfolder,f'{project_name}-LELCH.post.alyabin'), \
#                                number_of_blocks, alya_id_type)
#LEINV = read_alya_array(os.path.join(inputfolder,f'{project_name}-LEINV.post.alyabin'), \
#                                number_of_blocks, alya_id_type)

# Write geometry and variables

In [15]:
#blocks are mesh partitions
number_of_blocks = partitions.shape[0]
write_geometry(project_name, number_of_blocks)


for index, row in variable_info.iterrows():
    info = write_variable(row.field, row.iteration, number_of_blocks)
    variable_info.loc[index, 'time_real'] = info['time_real']
    variable_info.loc[index, 'time_int']= info['time_int']
    variable_info.loc[index, 'variabletype'] = info['variable_type']
    variable_info.loc[index, 'association'] = info['variable_association']


Reading array: 3 dim, 12030044 tuples

Block 0/199: 61061 tuples

Block 1/199: 60172 tuples

Block 2/199: 60531 tuples

Block 3/199: 59714 tuples

Block 4/199: 59787 tuples

Block 5/199: 60321 tuples

Block 6/199: 62375 tuples

Block 7/199: 60621 tuples

Block 8/199: 60369 tuples

Block 9/199: 61100 tuples

Block 10/199: 62291 tuples

Block 11/199: 62713 tuples

Block 12/199: 60751 tuples

Block 13/199: 59882 tuples

Block 14/199: 59789 tuples

Block 15/199: 60555 tuples

Block 16/199: 60170 tuples

Block 17/199: 60319 tuples

Block 18/199: 59632 tuples

Block 19/199: 59482 tuples

Block 20/199: 59792 tuples

Block 21/199: 59544 tuples

Block 22/199: 59756 tuples

Block 23/199: 58545 tuples

Block 24/199: 61938 tuples

Block 25/199: 61792 tuples

Block 26/199: 60020 tuples

Block 27/199: 59699 tuples

Block 28/199: 60066 tuples

Block 29/199: 60259 tuples

Block 30/199: 60235 tuples

Block 31/199: 60515 tuples

Block 32/199: 59042 tuples

Block 33/199: 59899 tuples

Block 34/199: 60320

Block 98/199: 329097 tuples

Block 99/199: 329128 tuples

Block 100/199: 329127 tuples

Block 101/199: 329124 tuples

Block 102/199: 329113 tuples

Block 103/199: 329146 tuples

Block 104/199: 329148 tuples

Block 105/199: 329482 tuples

Block 106/199: 325536 tuples

Block 107/199: 329142 tuples

Block 108/199: 329124 tuples

Block 109/199: 329211 tuples

Block 110/199: 329204 tuples

Block 111/199: 328507 tuples

Block 112/199: 328958 tuples

Block 113/199: 329230 tuples

Block 114/199: 333422 tuples

Block 115/199: 329203 tuples

Block 116/199: 329194 tuples

Block 117/199: 329213 tuples

Block 118/199: 329209 tuples

Block 119/199: 329190 tuples

Block 120/199: 329290 tuples

Block 121/199: 329200 tuples

Block 122/199: 329287 tuples

Block 123/199: 329135 tuples

Block 124/199: 328934 tuples

Block 125/199: 328849 tuples

Block 126/199: 328817 tuples

Block 127/199: 329123 tuples

Block 128/199: 328849 tuples

Block 129/199: 328850 tuples

Block 130/199: 328698 tuples

Block 131/19

Block 175/199: 329137 tuples

Block 176/199: 329136 tuples

Block 177/199: 329146 tuples

Block 178/199: 329196 tuples

Block 179/199: 329626 tuples

Block 180/199: 328241 tuples

Block 181/199: 328454 tuples

Block 182/199: 329045 tuples

Block 183/199: 328446 tuples

Block 184/199: 329796 tuples

Block 185/199: 329124 tuples

Block 186/199: 329641 tuples

Block 187/199: 329641 tuples

Block 188/199: 329641 tuples

Block 189/199: 329594 tuples

Block 190/199: 328969 tuples

Block 191/199: 329595 tuples

Block 192/199: 329110 tuples

Block 193/199: 329565 tuples

Block 194/199: 329114 tuples

Block 195/199: 329570 tuples

Block 196/199: 329547 tuples

Block 197/199: 329105 tuples

Block 198/199: 330407 tuples

../EP_141-INTRA-00000000.post.alyabin
Reading array: 1 dim, 12030044 tuples

Block 0/199: 61061 tuples

Block 1/199: 60172 tuples

Block 2/199: 60531 tuples

Block 3/199: 59714 tuples

Block 4/199: 59787 tuples

Block 5/199: 60321 tuples

Block 6/199: 62375 tuples

Block 7/199: 6

# Write ensight case file

In [16]:
case_file = f'{project_name}.ensi.case'
with open(os.path.join(outputfolder, case_file), 'w') as f:
    f.write('# Converted from Alya\n')
    f.write('# Ensight Gold Format\n')
    f.write('#\n')
    f.write(f'# Problem name: {project_name}\n')
    f.write('FORMAT\n')
    f.write('type: ensight gold\n')
    f.write('\n')
    f.write('GEOMETRY\n')
    f.write(f'model: 1 {project_name}.ensi.geo\n')
    f.write('\n')
    f.write('VARIABLE\n')
    
    variables = variable_info.field.unique();
    
    for varname in variables:
        df = variable_info[variable_info.field==varname].loc[0] #get one rectrod with this varibale
        line = f'{df.variabletype} per {df.association}: 1 {varname} {project_name}.ensi.{varname}-'+\
            '*'*iterationid_number_of_digits+'\n'       
        f.write(line)
        
    #this should be the same for all variables as I'm saving only one time series
    df_one_var =  variable_info[variable_info.field==variable_info.loc[0,'field']].sort_values(by='iteration');
    
    
    number_of_timesteps = variable_info[variable_info.field==variable_info.loc[0,'field']].shape[0]
    
    filename_increment = 0
    if df_one_var.shape[0]>1:
        filename_increment = df_one_var.loc[1,'iteration']-df_one_var.loc[0,'iteration']
    
    f.write('\n')
    f.write('TIME\n')
    f.write('time set: 1\n')
    f.write(f'number of steps: {number_of_timesteps}\n')
    f.write(f'filename start number: {df_one_var.iteration.min()+1}\n')
    f.write(f'filename increment: {filename_increment}\n') 
    f.write('time values:\n')
    f.write(str(df_one_var.time_real.as_matrix())[1:-1]+'\n')

In [57]:
a = np.dtype(np.int32)
b= np.array(a.name)
c = np.dtype(str(b))

In [59]:
c.type

numpy.int32